# 张量排序

- Sort/argsort
- Topk
- Top-5 Acc

In [1]:
import tensorflow as tf

In [3]:
# Sort, argsort

a = tf.random.shuffle(tf.range(5))

print(a)
print(tf.sort(a, direction='DESCENDING'))
print(tf.argsort(a, direction='DESCENDING'))

tf.Tensor([4 0 2 3 1], shape=(5,), dtype=int32)
tf.Tensor([4 3 2 1 0], shape=(5,), dtype=int32)
tf.Tensor([0 3 2 4 1], shape=(5,), dtype=int32)


In [5]:
idx = tf.argsort(a, direction='DESCENDING')
tf.gather(a, idx)

<tf.Tensor: id=56, shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0], dtype=int32)>

In [7]:
a = tf.random.uniform([3,3], maxval=10, dtype=tf.int32)

a, tf.sort(a), tf.sort(a, direction='DESCENDING')

(<tf.Tensor: id=77, shape=(3, 3), dtype=int32, numpy=
 array([[4, 0, 9],
        [6, 4, 7],
        [0, 2, 4]], dtype=int32)>,
 <tf.Tensor: id=89, shape=(3, 3), dtype=int32, numpy=
 array([[0, 4, 9],
        [4, 6, 7],
        [0, 2, 4]], dtype=int32)>,
 <tf.Tensor: id=98, shape=(3, 3), dtype=int32, numpy=
 array([[9, 4, 0],
        [7, 6, 4],
        [4, 2, 0]], dtype=int32)>)

In [13]:
# Top_k

a = tf.random.uniform([3,3], maxval=10, dtype=tf.int32)
# return top-k values and indices
res = tf.math.top_k(a, 2)

res, res.indices

(TopKV2(values=<tf.Tensor: id=146, shape=(3, 2), dtype=int32, numpy=
 array([[8, 6],
        [9, 1],
        [8, 4]], dtype=int32)>, indices=<tf.Tensor: id=147, shape=(3, 2), dtype=int32, numpy=
 array([[1, 0],
        [1, 0],
        [1, 0]], dtype=int32)>),
 <tf.Tensor: id=147, shape=(3, 2), dtype=int32, numpy=
 array([[1, 0],
        [1, 0],
        [1, 0]], dtype=int32)>)

In [20]:
# Top-k accuracy

# Prob:[0.1, 0.2, 0.3, 0.4]
# Label:[2]
# Only consider top-1 prediction: [3] 
# Only consider top-2 prediction: [3, 2]
# Only consider top-3 prediction: [3, 2, 1]

prob = tf.constant([[0.1,0.2,0.3],[0.2,0.7,0.1]])
target = tf.constant([2,0])

k_b = tf.math.top_k(prob,3).indices
k_b = tf.transpose(k_b, [1,0])
target = tf.broadcast_to(target, [3,2])

k_b, target

(<tf.Tensor: id=188, shape=(3, 2), dtype=int32, numpy=
 array([[2, 1],
        [1, 0],
        [0, 2]], dtype=int32)>,
 <tf.Tensor: id=190, shape=(3, 2), dtype=int32, numpy=
 array([[2, 0],
        [2, 0],
        [2, 0]], dtype=int32)>)

In [21]:
# Top-k Accuracy

def accuracy(output, target, topk=(1,)):
    maxk = max(topl)
    batch_size = target.shape[0]
    
    pred = tf.math.top_k(output, maxk).indices
    pred = tf.transpose(pred, perm=[1,0])
    target_ = tf.broadcast_to(target, pred.shape)
    correct = tf.equal(pred, target)
    
    res = []
    for k in topk:
        correct_k = tf.cast(tf.reshape(correct[:k], [-1], dtype=tf.float32))
        correct = tf.reduce_sum(correct_k)
        acc = float(correct_k /batch_size)
        res.append(acc)
    
    return res